In [1]:
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

from statsmodels.tsa.seasonal import MSTL
from statsmodels.tsa.seasonal import DecomposeResult
from tqdm import tqdm

In [2]:
data = []
data1 = pd.read_csv("store_sales_data1.csv")
data.append(data1)
data2 = pd.read_csv("store_sales_data2.csv")
data.append(data2)
data = pd.concat(data, ignore_index=True).drop_duplicates()
data['Date'] = pd.to_datetime(data['Date'])


In [3]:
sales_data = data

In [4]:
promo_data  = pd.read_excel('DataMining_Marketing.xlsx')

In [5]:
promo_data.head()
promo_data['Type'].unique()

array(['Free Delivery', 'Overall Discount', 'Product Discount',
       'Free Product', 'Points Bonus'], dtype=object)

In [6]:
sales_data = sales_data[sales_data['Date'] >= promo_data['Start Date'].min()].reset_index(drop=True)

In [7]:
free_del_data = promo_data[promo_data['Type'] == 'Free Delivery']
tot_discount_data = promo_data[promo_data['Type'] == 'Overall Discount']
prod_discount_data = promo_data[promo_data['Type'] == 'Product Discount']
free_prod_data = promo_data[promo_data['Type'] == 'Free Product']
points_bonus_data = promo_data[promo_data['Type'] == 'Points Bonus']
all_promo_data = [free_del_data,tot_discount_data,prod_discount_data,free_prod_data,points_bonus_data]

In [8]:
# sales_data["Promo_Active"] = sales_data["Date"].apply(lambda d: any((promo_data["Start Date"] <= d) & (promo_data["End Date"] >= d))).astype(int)

In [9]:
# sales_data["Promo_Type"] = sales_data["Date"].apply(
#     lambda d: promo_data.loc[
#         (promo_data["Start Date"] <= d) & (promo_data["End Date"] >= d), "Type"
#     ].values[0] if any((promo_data["Start Date"] <= d) & (promo_data["End Date"] >= d)) else None
# )

In [10]:
def apply_promo_columns(sales_df, promo_df, columns):
    """
    Adds multiple columns from promo_df to sales_df based on promo active dates.

    Parameters:
        sales_df (pd.DataFrame): Sales data with 'Date' column.
        promo_df (pd.DataFrame): Promotion data with 'Start Date' and 'End Date' columns.
        columns (list): List of column names from promo_df to add to sales_df.

    Returns:
        pd.DataFrame: Updated sales_df with new promo columns.
    """
    sales_df = sales_df.copy()
    for col in (columns):
        name =  promo_df['Type'].max() + "_" + col
        sales_df.loc[:, name] = sales_df.loc[:, "Date"].apply(lambda d: ", ".join(promo_df.loc[(promo_df["Start Date"] <= d) & (promo_df["End Date"] >= d), col].astype(str))
                                                     if any((promo_df["Start Date"] <= d) & (promo_df["End Date"] >= d)) else None)
    name =  promo_df['Type'].max() + "_"
    sales_df[name +"Promo_Active"] = sales_df["Date"].apply(lambda d: any((promo_df["Start Date"] <= d) & (promo_df["End Date"] >= d))).astype(int)
  
    return sales_df

In [11]:
columns = [ "Require Purchase", "Product"]
temp = apply_promo_columns(sales_df= sales_data,promo_df = free_del_data, columns = columns)
temp.head()

,store_id,Date,Total_Sales,Free Delivery_Require Purchase,Free Delivery_Product,Free Delivery_Promo_Active
0,14,2024-02-22,1397.62,Yes,Free Delivery,1
1,14,2024-02-23,1720.61,Yes,Free Delivery,1
2,14,2024-02-24,924.01,Yes,Free Delivery,1
3,14,2024-02-25,933.34,Yes,Free Delivery,1
4,14,2024-02-26,1052.80,None,None,0


In [12]:
all_data= []
for d in tqdm(all_promo_data):
  temp = apply_promo_columns(sales_df= sales_data,promo_df = d, columns = columns)
  sales_data = temp
sales_data.head()

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:38<00:00, 19.70s/it]


,store_id,Date,Total_Sales,Free Delivery_Require Purchase,Free Delivery_Product,Free Delivery_Promo_Active,Overall Discount_Require Purchase,Overall Discount_Product,Overall Discount_Promo_Active,Product Discount_Require Purchase,Product Discount_Product,Product Discount_Promo_Active,Free Product_Require Purchase,Free Product_Product,Free Product_Promo_Active,Points Bonus_Require Purchase,Points Bonus_Product,Points Bonus_Promo_Active
0,14,2024-02-22,1397.62,Yes,Free Delivery,1,No,All,1,None,None,0,None,None,0,None,None,0
1,14,2024-02-23,1720.61,Yes,Free Delivery,1,No,All,1,None,None,0,None,None,0,None,None,0
2,14,2024-02-24,924.01,Yes,Free Delivery,1,No,All,1,None,None,0,None,None,0,None,None,0
3,14,2024-02-25,933.34,Yes,Free Delivery,1,No,All,1,None,None,0,None,None,0,None,None,0
4,14,2024-02-26,1052.80,None,None,0,None,None,0,None,None,0,None,None,0,None,None,0


In [13]:
sales_data.reset_index(drop=True)

,store_id,Date,Total_Sales,Free Delivery_Require Purchase,Free Delivery_Product,Free Delivery_Promo_Active,Overall Discount_Require Purchase,Overall Discount_Product,Overall Discount_Promo_Active,Product Discount_Require Purchase,Product Discount_Product,Product Discount_Promo_Active,Free Product_Require Purchase,Free Product_Product,Free Product_Promo_Active,Points Bonus_Require Purchase,Points Bonus_Product,Points Bonus_Promo_Active
0,14,2024-02-22,1397.62,Yes,Free Delivery,1,No,All,1,None,None,0,None,None,0,None,None,0
1,14,2024-02-23,1720.61,Yes,Free Delivery,1,No,All,1,None,None,0,None,None,0,None,None,0
2,14,2024-02-24,924.01,Yes,Free Delivery,1,No,All,1,None,None,0,None,None,0,None,None,0
3,14,2024-02-25,933.34,Yes,Free Delivery,1,No,All,1,None,None,0,None,None,0,None,None,0
4,14,2024-02-26,1052.80,None,None,0,None,None,0,None,None,0,None,None,0,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19009,"2,104",2025-03-18,1781.74,None,None,0,None,None,0,None,None,0,None,None,0,"No, No","Points, Points",1
19010,"2,104",2025-03-19,1643.13,None,None,0,None,None,0,None,None,0,None,None,0,"No, No","Points, Points",1
19011,"2,104",2025-03-20,2086.75,None,None,0,None,None,0,None,None,0,None,None,0,"No, No","Points, Points",1
19012,"2,104",2025-03-21,3371.30,None,None,0,None,None,0,None,None,0,None,None,0,"No, No","Points, Points",1


In [14]:
sales_data.to_csv("sales_data_long.csv",index=False)